In [ ]:
import os
import subprocess
import numpy as np
import re
from PIL import Image
from datetime import datetime, timezone
from pyproj import Geod
from sgp4.api import Satrec, jday
import rasterio
from rasterio.control import GroundControlPoint
from rasterio.crs import CRS
from rasterio.transform import from_gcps

# === 1. Parametri ===
wavfile = "audio_137100000Hz_10-16-27_28-03-2024.wav"
output_png = "temp_decoded.png"

# === 2. Decodifica WAV → PNG (usa noaa-apt CLI tool) ===
# Viene generata un'immagine in scala di grigi
subprocess.run([
    "noaa-apt", "-o", output_png, wavfile
], check=True)

# === 3. Estrazione data/ora da nome file ===
match = re.search(r"_(\d{2})-(\d{2})-(\d{2})_(\d{2})-(\d{2})-(\d{4})\.wav$", wavfile)
if not match:
    raise ValueError("Formato nome file .wav non riconosciuto")

hh, mm, ss, day, month, year = map(int, match.groups())
start_time = datetime(year, month, day, hh, mm, ss, tzinfo=timezone.utc)
print(f"🕒 Inizio acquisizione: {start_time.isoformat()}")

# === 4. Download TLE NOAA-19 (137.1 MHz) ===
import requests
tle_url = "https://celestrak.com/NORAD/elements/weather.txt"
resp = requests.get(tle_url)
tle_lines = resp.text.splitlines()
for i, line in enumerate(tle_lines):
    if "NOAA 19" in line.upper():
        tle1 = tle_lines[i+1].strip()
        tle2 = tle_lines[i+2].strip()
        break
else:
    raise RuntimeError("TLE NOAA-19 non trovato")

sat = Satrec.twoline2rv(tle1, tle2)
geod = Geod(ellps="WGS84")

# === 5. Caricamento PNG come array immagine ===
image = Image.open(output_png).convert("L")
data = np.array(image)
H, W = data.shape

# === 6. Calcolo coordinate sub-satellite ===
timestamps = [start_time.timestamp() + i * 0.5 for i in range(H)]
def get_subsatellite_point(t):
    dt = datetime.utcfromtimestamp(t)
    jd, fr = jday(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second + dt.microsecond / 1e6)
    e, r, _ = sat.sgp4(jd, fr)
    if e != 0:
        return None
    x, y, z = r
    r_eq = (x**2 + y**2)**0.5
    lat = np.degrees(np.arctan2(z, r_eq))
    lon = np.degrees(np.arctan2(y, x))
    return lat, lon

swath_m = 2800000  # 2800 km
def interpolate_line(lat, lon):
    return [geod.fwd(lon, lat, 90, ((i - W/2) / W) * swath_m)[0:2][::-1] for i in range(W)]

all_coords = []
for t in timestamps:
    pt = get_subsatellite_point(t)
    if pt is None:
        all_coords.append([(None, None)] * W)
        continue
    lat, lon = pt
    all_coords.append(interpolate_line(lat, lon))

# === 7. Scrivi GCP e salva GeoTIFF ===
gcps = []
spacing_y = max(H // 20, 1)
spacing_x = max(W // 20, 1)
for y in range(0, H, spacing_y):
    for x in range(0, W, spacing_x):
        lat, lon = all_coords[y][x]
        if lat is not None:
            gcps.append(GroundControlPoint(row=y, col=x, x=lon, y=lat))

with rasterio.open(
    "APT-georef.tif", "w",
    driver="GTiff", width=W, height=H, count=1, dtype=data.dtype,
    crs="EPSG:4326", transform=from_gcps(gcps)
) as dst:
    dst.write(data, 1)
    dst.gcps = (gcps, "EPSG:4326")

print("✅ GeoTIFF salvato come 'APT-georef.tif'")
